In [1]:
import pandas as pd
import numpy as np

import os
from datetime import timedelta, date, datetime
from sqlalchemy import create_engine
from sqlalchemy.types import NVARCHAR, DATE, FLOAT, VARCHAR, DATETIME

import config

data_dir = "data"

engine = create_engine(
    f"mysql+pymysql://{config.sql_credentials['user']}:{config.sql_credentials['password']}@{config.sql_credentials['host']}:{config.sql_credentials['port']}/{config.sql_credentials['db']}",
    pool_pre_ping=True)

conn = engine.connect()




## Creating SMS table

In [41]:
# import csvs

data = pd.DataFrame()

df_names = ['carer_df_1909_2001',
            'carer_df_2001_2002',
            'carer_df_2002_2006',
            'carer_df_2006_2101',
            'carer_df_2101_2103',
            'carer_df_2103_2108']

for f in df_names:

    filename = f'../data/bulk_messages/output_files/{f}.csv'
    if os.path.isfile(filename):
        print(f'{f} is being appended')

        carer_df = pd.read_csv(filename, parse_dates=['sent_at'])
        data = data.append(carer_df)

print(data.shape)
data.drop_duplicates(subset=['placement_ad_id','carer_id','sent_at'],inplace = True) # more than one number per carer?
print(data.shape)


carer_df_1909_2001 is being appended
carer_df_2001_2002 is being appended
carer_df_2002_2006 is being appended
carer_df_2006_2101 is being appended
carer_df_2101_2103 is being appended
carer_df_2103_2108 is being appended
(1232843, 4)
(1174932, 4)


In [42]:
print(data.sent_at.min(),data.sent_at.max())

2019-09-17 13:12:00 2021-08-17 18:11:00


In [43]:
data['sms_type'] = 'customListSent'

data['placement_ad_id_carer_id_sent_at'] = data['placement_ad_id'] + data['carer_id'] + data['sent_at'].apply(lambda x:str(x))
print(data['placement_ad_id_carer_id_sent_at'].nunique()==data.shape[0])

True


In [46]:

dtypes = {
    'sent_at': DATETIME,
}

data_columns = [
                'carer_id',
                'placement_ad_id',
                'sent_at',
                'sms_type',
                'placement_ad_id_carer_id_sent_at'
                ]

data = data[data_columns]

data.head()

,carer_id,placement_ad_id,sent_at,sms_type,placement_ad_id_carer_id_sent_at
0,p258326082,plad_pl_s03iuz2MBGtu_UJ2Y-7f6e6167-c776-447a-a...,2019-09-17 13:12:00,customListSent,plad_pl_s03iuz2MBGtu_UJ2Y-7f6e6167-c776-447a-a...
1,p91963657,plad_pl_s03iuz2MBGtu_UJ2Y-7f6e6167-c776-447a-a...,2019-09-17 13:12:00,customListSent,plad_pl_s03iuz2MBGtu_UJ2Y-7f6e6167-c776-447a-a...
2,p54540830,plad_pl_s03iuz2MBGtu_UJ2Y-7f6e6167-c776-447a-a...,2019-09-17 13:12:00,customListSent,plad_pl_s03iuz2MBGtu_UJ2Y-7f6e6167-c776-447a-a...
3,p73830676,plad_pl_s03iuz2MBGtu_UJ2Y-7f6e6167-c776-447a-a...,2019-09-17 13:12:00,customListSent,plad_pl_s03iuz2MBGtu_UJ2Y-7f6e6167-c776-447a-a...
4,p96258457,plad_pl_s03iuz2MBGtu_UJ2Y-7f6e6167-c776-447a-a...,2019-09-17 13:12:00,customListSent,plad_pl_s03iuz2MBGtu_UJ2Y-7f6e6167-c776-447a-a...


In [50]:
print(f'Starting writing table at {datetime.now()}')

data[data_columns].sort_values(['sent_at','placement_ad_id']).to_sql(
    'live_MO_CUSTOM_LIST_MATCH_REQUEST_CARER_SMS',
    con=conn,
    schema="elder_live",
    if_exists="replace",
    index=False,
    dtype= dtypes
)

conn.execute(
        'ALTER TABLE `elder_live`.`live_MO_CUSTOM_LIST_MATCH_REQUEST_CARER_SMS` CHANGE COLUMN `placement_ad_id` `placement_ad_id` VARCHAR(128) NOT NULL;')

conn.execute(
        'ALTER TABLE `elder_live`.`live_MO_CUSTOM_LIST_MATCH_REQUEST_CARER_SMS` CHANGE COLUMN `sent_at` `sent_at` DATETIME NOT NULL;')

conn.execute(
        'ALTER TABLE `elder_live`.`live_MO_CUSTOM_LIST_MATCH_REQUEST_CARER_SMS` CHANGE COLUMN `carer_id` `carer_id` VARCHAR(16) NOT NULL;')

conn.execute(
        'ALTER TABLE `elder_live`.`live_MO_CUSTOM_LIST_MATCH_REQUEST_CARER_SMS` CHANGE COLUMN `sms_type` `sms_type` VARCHAR(32) NOT NULL;')

conn.execute(
        'ALTER TABLE `elder_live`.`live_MO_CUSTOM_LIST_MATCH_REQUEST_CARER_SMS` ADD UNIQUE INDEX `placement_ad_id_sent_at_carer_id` (`placement_ad_id` ASC, `sent_at` ASC, `carer_id` ASC);')

conn.execute(
        'ALTER TABLE `elder_live`.`live_MO_CUSTOM_LIST_MATCH_REQUEST_CARER_SMS` ADD UNIQUE INDEX `sent_at_placement_ad_id_carer_id` (`sent_at` ASC,`placement_ad_id` ASC, `carer_id` ASC);')

conn.execute(
        'ALTER TABLE `elder_live`.`live_MO_CUSTOM_LIST_MATCH_REQUEST_CARER_SMS` ADD UNIQUE INDEX `carer_id_sent_at_placement_ad_id` (`carer_id` ASC, `sent_at` ASC,`placement_ad_id` ASC);')

conn.execute(
        'ALTER TABLE `elder_live`.`live_MO_CUSTOM_LIST_MATCH_REQUEST_CARER_SMS` ADD UNIQUE INDEX `sent_at_carer_id_placement_ad_id` (`sent_at` ASC,`carer_id` ASC,`placement_ad_id` ASC);')


print(f'Completing writing table at {datetime.now()}')


Starting writing table at 2021-09-03 09:23:49.531803
Completing writing table at 2021-09-03 09:28:53.306041


## Creating  placement views table

In [2]:
viewed_placements = pd.read_csv('../data/viewed_placements/viewed_placements.csv')

In [15]:
from random import choice
from string import ascii_uppercase

viewed_placements['unique_id'] = viewed_placements['placement_ad_id'] + "_" + viewed_placements['carer_id'] + "_" + \
                                 viewed_placements['date'].apply(lambda x:str(x) + '_' +
                                  ''.join(choice(ascii_uppercase) for i in range(4)))

print(viewed_placements.shape[0] == viewed_placements.unique_id.nunique())

MUOK


In [22]:
dtypes = {
    'date': DATE,
}

data_columns = [
                'carer_id',
                'placement_ad_id',
                'date',
                'unique_id'
                ]

viewed_placements = viewed_placements[data_columns]

viewed_placements.head()

,carer_id,placement_ad_id,date,unique_id
0,p23128636,plad_pl_s0dAkDzkUx6S_mPu4-7fc95ad1-0a4c-4407-8...,2019-08-01,plad_pl_s0dAkDzkUx6S_mPu4-7fc95ad1-0a4c-4407-8...
1,p108185489,plad_pl_s0lKdHg1XPOA_BNv0-083e29c0-bb16-4730-b...,2019-08-01,plad_pl_s0lKdHg1XPOA_BNv0-083e29c0-bb16-4730-b...
2,p108185489,plad_pl_s0ZWbtH4t9ch_wol3-161e6f76-9db6-43af-b...,2019-08-01,plad_pl_s0ZWbtH4t9ch_wol3-161e6f76-9db6-43af-b...
3,p187627633,plad_pl_s03Ms15xHfIl_2NRK-80e9ad14-a5ca-4e4c-8...,2019-08-01,plad_pl_s03Ms15xHfIl_2NRK-80e9ad14-a5ca-4e4c-8...
4,p226451629,plad_pl_s0jdbDkIxpyq_kmXh-1b365ec2-fe66-41ae-9...,2019-08-01,plad_pl_s0jdbDkIxpyq_kmXh-1b365ec2-fe66-41ae-9...


In [31]:
print(f'Starting writing table at {datetime.now()}')

viewed_placements[data_columns].sort_values(['date','carer_id']).to_sql(
    'live_MO_AMPLITUDE_VIEWED_PLACEMENTS',
    con=conn,
    schema="elder_live",
    if_exists="replace",
    index=False,
    dtype= dtypes
)

conn.execute(
        'ALTER TABLE `elder_live`.`live_MO_AMPLITUDE_VIEWED_PLACEMENTS` CHANGE COLUMN `placement_ad_id` `placement_ad_id` VARCHAR(128) NOT NULL;')

conn.execute(
        'ALTER TABLE `elder_live`.`live_MO_AMPLITUDE_VIEWED_PLACEMENTS` CHANGE COLUMN `date` `date` DATE NOT NULL;')

conn.execute(
        'ALTER TABLE `elder_live`.`live_MO_AMPLITUDE_VIEWED_PLACEMENTS` CHANGE COLUMN `carer_id` `carer_id` VARCHAR(16) NOT NULL;')

conn.execute(
        'ALTER TABLE `elder_live`.`live_MO_AMPLITUDE_VIEWED_PLACEMENTS` CHANGE COLUMN `unique_id` `unique_id` VARCHAR(128) NOT NULL;')

conn.execute(
        'ALTER TABLE `elder_live`.`live_MO_AMPLITUDE_VIEWED_PLACEMENTS` ADD UNIQUE INDEX `unique_id` (`unique_id` ASC);')

conn.execute(
        'ALTER TABLE `elder_live`.`live_MO_AMPLITUDE_VIEWED_PLACEMENTS` ADD INDEX `date_placement_ad_id_carer_id` (`date` ASC,`placement_ad_id` ASC, `carer_id` ASC);')

conn.execute(
        'ALTER TABLE `elder_live`.`live_MO_AMPLITUDE_VIEWED_PLACEMENTS` ADD INDEX `carer_id_date_placement_ad_id` (`carer_id` ASC, `date` ASC,`placement_ad_id` ASC);')

conn.execute(
        'ALTER TABLE `elder_live`.`live_MO_AMPLITUDE_VIEWED_PLACEMENTS` ADD INDEX `placement_ad_id_date_carer_id` (`carer_id` ASC,`placement_ad_id` ASC,`date` ASC);')


conn.execute(
        'ALTER TABLE `elder_live`.`live_MO_AMPLITUDE_VIEWED_PLACEMENTS` ADD INDEX `date` (`date` ASC);')


print(f'Completing writing table at {datetime.now()}')

#%

Starting writing table at 2021-09-14 10:15:33.015907
Completing writing table at 2021-09-14 10:15:53.232093
